In [185]:
import requests
import time
import datetime
import math
import re
import lxml.html

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img

from pandas.io.json import json_normalize
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, datetime, timedelta

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
## 한글 폰트
import matplotlib.font_manager as fm
import matplotlib as mpl

font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')

for font in font_list:
    if 'Nanum' in font:
        print(font)
        
print()
print(mpl.matplotlib_fname())

plt.rcParams["font.family"] = 'NanumGothic'

## 마이너스 부호
mpl.rcParams['axes.unicode_minus'] = False

/Library/Fonts/NanumMyeongjoExtraBold.ttf
/Library/Fonts/NanumSquareRoundB.ttf
/Library/Fonts/NanumGothic.ttf
/Library/Fonts/NanumSquareRegular.ttf
/Library/Fonts/NanumBarunpenRegular.ttf
/Library/Fonts/NanumSquareRoundEB.ttf
/Library/Fonts/NanumBarunGothic.ttf
/Library/Fonts/NanumPen.ttf
/Library/Fonts/NanumBarunGothicUltraLight.ttf
/Library/Fonts/NanumBarunGothicLight.ttf
/Library/Fonts/NanumBrush.ttf
/Library/Fonts/NanumGothicLight.ttf
/Library/Fonts/NanumSquareLight.ttf
/Library/Fonts/NanumGothicBold.ttf
/Library/Fonts/NanumBarunGothicBold.ttf
/Library/Fonts/NanumBarunpenBold.ttf
/System/Library/AssetsV2/com_apple_MobileAsset_Font6/5c10807185ddbca859bc0b3ecefe55ebb1985aa9.asset/AssetData/NanumMyeongjo.ttc
/Library/Fonts/NanumSquareRoundR.ttf
/Library/Fonts/NanumSquareBold.ttf
/System/Library/AssetsV2/com_apple_MobileAsset_Font6/3b0bb049516d6dd825e1cd8731587ca7415517ff.asset/AssetData/NanumScript.ttc
/Library/Fonts/NanumSquareExtraBold.ttf
/System/Library/AssetsV2/com_apple_MobileAs

# INVEN e-sports 기록실

In [244]:
season_dict = {
    '2020_spring':137,
    '2020_spring_ps':138,
    '2021_summer':141,
    '2021_summer_ps':142,
    '2021_spring':147,
    '2021_spring_ps':148,
    '2021_summer':160,
    '2021_summer_ps':161,
}

season_date_dict = {
    '2021_spring':['2021.01.13','2021.03.28'],
    '2021_spring_ps':['2021.03.31','2021.04.11'],
    '2021_summer':['2021.06.09','2021.08.15'],
    '2021_summer_ps':['2021.08.18','2021.08.29']
}

teamname_dict = {
    '':'',
    'T1':'T1',
    'DWGKIA':'DWG+KIA',
    'GENG':'Gen.G+eSports',
    'HLE':'Hanwha+Life+Esports'
}

teamplayer_dict = {
    'T1':['Canna', 'Cuzz', 'Oner', 'Faker', 'Teddy', 'Gumayusi', 'Keria'],
    'DWGKIA':['Khan', 'Canyon', 'Showmaker', 'Ghost', 'Beryl'],
    'GENG':['Rascal', 'Burdol', 'Clid', 'Bdd', 'Ruler', 'Life'],
    'HLE':['Morgan', 'Dudu', 'Willer', 'Chovy', 'Deft', 'Vsta']
}

pickban_list = ['blue_ban_1', 'blue_ban_2', 'blue_ban_3', 'blue_ban_4', 'blue_ban_5', 'blue_pick_1', 'blue_pick_2', 'blue_pick_3', 'blue_pick_4', 'blue_pick_5', \
                'red_ban_1', 'red_ban_2', 'red_ban_3', 'red_ban_4', 'red_ban_5', 'red_pick_1', 'red_pick_2', 'red_pick_3', 'red_pick_4', 'red_pick_5']
    

invel_champ_url = 'https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol'
inven_player_url = 'https://lol.inven.co.kr/dataninfo/match/playerList.php?iskin=lol'
inven_game_url = 'https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol'

## 챔피언 통산 기록

In [19]:
def get_inven_champ_record(teamname='', season='2021_summer'):
    
    url = f'{invel_champ_url}&category2={season_dict[season]}&teamName={teamname_dict[teamname]}'
    print(url)

    html = urlopen(url)
    bsObject = BeautifulSoup(html, 'html.parser')
    contents = bsObject.select_one('#lolDbMatchPlayerTotal > div.listTable')

    columns = []
    for th in contents.find_all('th'):
        columns.append(th.get_text())

    inven_champ_df = pd.DataFrame(columns=columns)

    data = []
    for i, td in enumerate(contents.find_all('td')):

        #챔피언 영어이름 추출
        if td.get('class') == ['left']:
            text = td.find('img').get('src').split('/')[-1].split('_')[0]
        else:
            text = td.get_text()
            text = text.replace('%','')
            if text == '－': text = np.nan
            try:
                text = int(text)
            except ValueError as e:
                text = float(text)

        data.append(text)

        if i % 13 == 12:
            temp_df = pd.DataFrame(columns=columns, data=[data])
            inven_champ_df = inven_champ_df.append(temp_df)
            data = []

    inven_champ_df.drop(columns=['순위'], inplace=True)
    inven_champ_df.to_csv(f"inven_data/inven_champ_{season}{teamname if teamname=='' else '_'+teamname}.csv")
    inven_champ_df.head()

In [77]:
for season, _ in season_dict.items():
    for teamname, _ in teamname_dict.items():

        get_inven_champ_record(teamname, season)

https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=147&teamName=
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=147&teamName=T1
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=147&teamName=DWG+KIA
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=147&teamName=Gen.G+eSports
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=147&teamName=Hanwha+Life+Esports
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=148&teamName=
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=148&teamName=T1
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=148&teamName=DWG+KIA
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=148&teamName=Gen.G+eSports
https://lol.inven.co.kr/dataninfo/match/champTotal.php?iskin=lol&category2=148&teamName=Hanwha+Life+Esports
https://lol.inven.co.kr/data

## 선수 개인 기록

In [147]:
def get_inven_player_record(player, season):

    url = f'{inven_player_url}&category2={season_dict[season]}&playerName={player}&champ=0'
    print(url)

    html = urlopen(url)
    bsObject = BeautifulSoup(html, 'html.parser')
    contents = bsObject.select_one('#lolDbMatchPlayerList > div:nth-child(8)')
    
    if not contents.find_all('th'):
        contents = bsObject.select_one('#lolDbMatchPlayerList > div.listTable')
    
    columns = []
    for th in contents.find_all('th'):
        columns.append(th.get_text())

    inven_player_df = pd.DataFrame(columns=columns)

    data = []
    for i, td in enumerate(contents.find_all('td')):

        #챔피언 영어이름 추출
        if i % 11 == 3:
            text = td.find('img').get('src').split('/')[-1].split('_')[0]
        else:
            text = td.get_text()
            text = text.replace('%','')

        data.append(text)

        if i % 11 == 10:
            temp_df = pd.DataFrame(columns=columns, data=[data])
            inven_player_df = inven_player_df.append(temp_df)
            data = []

    inven_player_df.drop(columns=['소환주문'], inplace=True)
    inven_player_df = inven_player_df.astype({'K':int, 'D':int, 'A':int, 'KDA':float, '킬관여율':float})
    inven_player_df.to_csv(f"inven_data/inven_champ_{season}_{player}.csv")
    inven_player_df.head()

In [ ]:
for season, _ in season_dict.items():
    for team, players in teamplayer_dict.items():
        print('===', team, '===')

        for player in players:
            get_inven_player_record(player, season)

        print()

## 픽밴 기록

In [240]:
def get_inven_game_record(season):
    
    columns=['date', 'blue_team', 'blue_team_win', 'red_team', 'red_team_win']+pickban_list
    
    game_df = pd.DataFrame(columns=columns)
    
    start_date = datetime.strptime(season_date_dict[season][0], '%Y.%m.%d')
    end_date = datetime.strptime(season_date_dict[season][1], '%Y.%m.%d')
    
    current_date = start_date
    
    while current_date <= end_date:
        week_later = current_date + timedelta(days=7)
        
        url = f'{inven_game_url}&category2={season_dict[season]}&startDate={current_date.strftime("%Y-%m-%d")}&endDate={week_later.strftime("%Y-%m-%d")}'
        print(url)
        
        html = urlopen(url)
        bsObject = BeautifulSoup(html, 'html.parser')
        contents = bsObject.select_one('#lolDbMatchTeamList')
        
        matches = contents.find_all('div', 'listFrame')
        for match in matches:
            match_date = match.find('div', 'date').get_text()
            
            # blue team
            blue_side = match.find('div', 'wTeam')
            blue_team_name = blue_side.find('a', 'teamname').get_text()
            blue_team_win = blue_side.find('div', 'color1 tx5').get_text()
            
            # red team
            red_side = match.find('div', 'lTeam')
            red_team_name = red_side.find('a', 'teamname').get_text()
            red_team_win = red_side.find('div', 'color2 tx5').get_text()
          
            data = [match_date, blue_team_name, blue_team_win, red_team_name, red_team_win]
            
            for side in [blue_side, red_side]:
                
                bans = side.find('ul', 'ban')
                for ban in bans.find_all('li'):
                    if ban.find('span'):
                        pass
                    
                    elif ban.find('img'):
                        champ = ban.find('img').get('src').split('/')[-1].split('_')[0]
                        data.append(champ)
                        
                    else:
                        data.append('-')
                    
                picks = side.find('ul', 'pick')
                for pick in picks.find_all('li'):
                    champ = pick.find('img').get('src').split('/')[-1].split('_')[0]
                    data.append(champ)
                    
            try:
                temp_df = pd.DataFrame(columns=columns, data=[data])
            except Exception as e:
                print(data)
                
            game_df = game_df.append(temp_df)
            data = []
            
        current_date = week_later
        
    game_df.to_csv(f"inven_data/inven_game_pickban_{season}.csv")

In [245]:
for season, _ in season_date_dict.items():
    print('===',season,'===')
    get_inven_game_record(season)

=== 2021_spring ===
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-01-13&endDate=2021-01-20
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-01-20&endDate=2021-01-27
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-01-27&endDate=2021-02-03
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-02-03&endDate=2021-02-10
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-02-10&endDate=2021-02-17
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-02-17&endDate=2021-02-24
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-02-24&endDate=2021-03-03
https://lol.inven.co.kr/dataninfo/match/teamList.php?iskin=lol&category2=147&startDate=2021-03-03&endDate=2021-03-10
https://lol.inven.co.kr/dataninfo/match/team

In [89]:
df = pd.read_csv('inven_data/inven_champ_2021_summer_T1.csv')[['챔피언', order]].set_index('챔피언').rename(columns={order:'summer'})
df2 = pd.read_csv('inven_data/inven_champ_2021_summer_ps_T1.csv')[['챔피언', order]].set_index('챔피언').rename(columns={order:'summer_ps'})

In [106]:
df = pd.read_csv('inven_data/inven_champ_2021_summer_T1.csv').set_index('챔피언')
df2 = pd.read_csv('inven_data/inven_champ_2021_summer_ps_T1.csv').set_index('챔피언')

In [167]:
df1 = pd.read_csv('inven_data/inven_champ_2021_spring_Cuzz.csv').set_index('챔피언')
df2 = pd.read_csv('inven_data/inven_champ_2021_summer_ps_Cuzz.csv').set_index('챔피언')

In [168]:
df = pd.concat([df1, df2])
df['승'] = df['승패'].apply(lambda x: 1 if x == '승' else 0)
df['패'] = df['승패'].apply(lambda x: 1 if x == '패' else 0)
df_mean = df.groupby(by='챔피언').mean().round(2)
# [['K', 'D', 'A', 'KDA', '킬관여율']]
# df_sum = df.groupby(by='챔피언').sum()[['승', '패']].astype(int)
# df_sum['경기수'] = df_sum['승'] + df_sum['패']
# df_sum['승률'] = (df_sum['승']/df_sum['경기수']*100).round(2)

In [173]:
df1['K']

챔피언
lillia      2
Volibear    2
Udyr        5
Nidalee     3
Hecarim     2
Nidalee     7
Udyr        6
Hecarim     2
Hecarim     2
Olaf        2
JarvanIV    1
Udyr        6
Taliyah     3
Name: K, dtype: int64

In [110]:
df2.head()

,Unnamed: 0,밴,밴순서,밴비율,픽,픽순서,픽비율,밴+픽,밴픽률,승리,패배,승률
챔피언,,,,,,,,,,,,
Gragas,0,0,NaN,0.0,1,3.00,9.1,1,9.1,0,1,0.0
Gwen,0,0,NaN,0.0,3,3.33,27.3,3,27.3,1,2,33.3
Gnar,0,0,NaN,0.0,2,3.50,18.2,2,18.2,2,0,100.0
Nidalee,0,0,NaN,0.0,3,2.00,27.3,3,27.3,2,1,66.7
Diana,0,2,3.5,18.2,0,NaN,0.0,2,18.2,0,0,0.0


In [112]:
df_add = df.add(df2, fill_value=0)
df_add

,Unnamed: 0,밴,밴순서,밴비율,픽,픽순서,픽비율,밴+픽,밴픽률,승리,패배,승률
챔피언,,,,,,,,,,,,
Akali,0.0,10.0,2.50,22.7,2.0,5.00,5.9,12.0,27.3,0.0,2.0,0.0
Alistar,0.0,4.0,5.00,9.1,2.0,10.00,11.6,6.0,20.5,1.0,1.0,100.0
Aphelios,0.0,1.0,4.00,9.1,4.0,6.00,16.8,5.0,25.0,3.0,1.0,166.7
Ashe,0.0,3.0,3.67,27.3,4.0,7.00,29.5,7.0,50.0,4.0,0.0,200.0
Azir,0.0,3.0,4.00,20.5,7.0,6.92,42.6,10.0,56.9,4.0,3.0,116.7
...,...,...,...,...,...,...,...,...,...,...,...,...
Zoe,0.0,1.0,5.00,2.3,0.0,NaN,0.0,1.0,2.3,0.0,0.0,0.0
kaisa,0.0,3.0,4.33,6.8,1.0,5.00,2.4,4.0,9.1,0.0,1.0,0.0
ornn,0.0,1.0,5.00,9.1,0.0,NaN,0.0,1.0,9.1,0.0,0.0,0.0


In [116]:
df_add.loc['Gwen']

Unnamed: 0      0.00
밴               1.00
밴순서             5.00
밴비율             2.30
픽               6.00
픽순서             5.33
픽비율            34.30
밴+픽             7.00
밴픽률            36.40
승리              4.00
패배              2.00
승률            133.30
Name: Gwen, dtype: float64

In [122]:
df_add.loc['Gwen', '밴']

1.0

In [121]:
pd.concat([df, df2]).groupby(level=0).mean().loc['Gwen']

Unnamed: 0     0.000
밴              0.500
밴순서            5.000
밴비율            1.150
픽              3.000
픽순서            2.665
픽비율           17.150
밴+픽            3.500
밴픽률           18.200
승리             2.000
패배             1.000
승률            66.650
Name: Gwen, dtype: float64

In [95]:
dfdf = pd.concat([df, df2, df], axis=1)
dfdf['mean'] = dfdf.mean(axis=1)
dfdf

,밴비율,밴비율,밴비율,mean
Galio,0.0,NaN,0.0,0.000000
Gangplank,4.5,NaN,4.5,4.500000
Gragas,0.0,0.0,0.0,0.000000
Graves,0.0,NaN,0.0,0.000000
Gwen,2.3,0.0,2.3,1.533333
...,...,...,...,...
TwistedFate,0.0,18.2,0.0,6.066667
Rell,NaN,0.0,NaN,0.000000
Vayne,NaN,0.0,NaN,0.000000
ornn,NaN,9.1,NaN,9.100000


In [99]:
dfdf.sort_values('mean', ascending=False, inplace=True)

In [105]:
list(dfdf.columns)

['밴비율', '밴비율', '밴비율', 'mean']

In [175]:
{x:int for x in ['1','2']}

{'1': int, '2': int}